# 

# Import

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import unicodedata # pour enlever les accents

# 1. Import de notre selection

In [3]:
selection = pd.read_csv("Etape_3_selection.csv")  #join.csv est notre dataset final

# 2. Préparation du model

## 2.1. Choix de X et uniformisation

In [4]:
X = selection[selection.select_dtypes("number").columns]
scaler = StandardScaler()
scaler.fit(X)
X_scaler = scaler.transform(X)
X_scaler = pd.DataFrame(X_scaler, columns=selection.select_dtypes("number").columns)

## 2.2. Entraînement  du model

In [5]:
# Nous l'initialisons et l'entraînons pour rechercher les 6 voisins

distance = NearestNeighbors(n_neighbors=6).fit(X_scaler)

## 2.3. Fonctions pour l'automatisation et la sortie des résultats

### 2.3.1. Fonction d'affichage

In [6]:
def display(search, model, data, x_scale, titre):
    """
    input: search : array of funded index
           model : trained model
           data : the dataframe with movies
           x_scale : X value
           titre : title from input user
    """
    x_scaler = x_scale
    title = titre
    search_index = search.index
    
    if len(search) == 1:
        proches_title = model.kneighbors(x_scaler.iloc[search_index])
        print(f"""\nLes 5 films que vous nous recommandons pour \
\"{data.iloc[search_index[0]]['title_fr']}\" sorti en \
{int(data.iloc[search_index]['startYear'])} :\n""")
        
        for x in range(1, len(proches_title[1][0])):
            result = data.iloc[proches_title[1][0][x]]
            print(f'\tFilm numéro {x} :', result.loc["title_fr"] if result.loc["title_fr"] != '~'
                  else result.loc["originalTitle"], f'de {int(result.loc["startYear"])}')
            print()
        return True
    elif len(search) > 1:
        print(f"\nNous avons trouvé {len(search)} films indiquez le film souhaité :\n")

        for choix in range(len(search)):
            print(f"""Choix {choix + 1} : {data.iloc[search.index[choix]]['title_fr']} \
sorti en {int(data.iloc[search.index[choix]]['startYear'])}""")
            
        choix_user = int(input(f"\nVeuillez sélectionner un film entre 1 et {len(search)} : "))
        # Forget verification on choix_user
        
        proches_title = model.kneighbors([x_scaler.iloc[search_index[choix_user - 1]]])
        print(f"""\nLes 5 films que vous nous recommandons pour \
\"{search.iloc[choix_user - 1]['title_fr']}\" de {int(search.iloc[choix_user - 1]['startYear'])} : \n""")
      
        for x in range(1, len(proches_title[1][0])):
            result = data.iloc[proches_title[1][0][x]]
            print(f'\tFilm numéro {x} :', result.loc["title_fr"] if result.loc['title_fr'] != '~'
else f"{result.loc['originalTitle']} (Titre en VO)",f'de {int(result.loc["startYear"])} ')
            print()
        return True
    else:
        return False
        

### 2.3.2. Fonction pour avoir des recommandations

In [8]:
def reco_film(data, x_scaler, model):
    """ 
        data = df selection de films features
        x_scaler = Features scaled
        model = Trained model
        return = nom 5 films proches voisins 
    
    """
    title_input = input("Veuillez entrer un titre de film complet : ")
    distance = model
    # Transform input to lower, strip and without accent
    
    nfkd_form = unicodedata.normalize('NFKD', title_input).strip()
    title = u"".join([c for c in nfkd_form if not unicodedata.combining(c)]).strip().lower()
    search = data.loc[data['title'] == title]
    if len(search) > 0:
        if(display(search, distance, data, x_scaler, title)):
            return f"Toute l'équipe de WildSalto vous souhaite Bon visionnage !!"

    else:
        search_2 = data.loc[data['title'].str.contains(title)]
        if(display(search_2, distance, data, x_scaler, title)):
            return f"Toute l'équipe de WildSalto vous souhaite Bon visionnage !!"
        else:
            return str(f'Le film "{title}" ne se trouve pas dans notre sélection :(')

In [11]:
reco_film(data=selection, x_scaler=X_scaler, model=distance)


Nous avons trouvé 41 films indiquez le film souhaité :

Choix 1 : Faster, Pussycat! Kill! Kill! sorti en 1965
Choix 2 : Astérix et Cléopâtre sorti en 1968
Choix 3 : Le clan des Mac Masters sorti en 1970
Choix 4 : Le monastère de Shaolin sorti en 1974
Choix 5 : Les Douze travaux d'Astérix sorti en 1976
Choix 6 : Astérix et la surprise de César sorti en 1985
Choix 7 : Astérix chez les Bretons sorti en 1986
Choix 8 : Kung-fu master! sorti en 1988
Choix 9 : Astérix et le coup du menhir sorti en 1989
Choix 10 : The Master sorti en 1992
Choix 11 : Tai Chi Master sorti en 1993
Choix 12 : Astérix et les indiens sorti en 1994
Choix 13 : Astérix & Obélix contre César sorti en 1999
Choix 14 : The Poison Tasters sorti en 1995
Choix 15 : Astérix & Obélix : Mission Cléopâtre sorti en 2002
Choix 16 : The Master of Disguise sorti en 2002
Choix 17 : Master and Commander: De l'autre côté du monde sorti en 2003
Choix 18 : The Yin-Yang Master sorti en 2001
Choix 19 : Inglourious Basterds sorti en 2009
Ch

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


"Toute l'équipe de WildSalto vous souhaite Bon visionnage !!"